# 一个关于LSTM生成歌词的练习

新手上路，这是一个练习笔记，其中可能存在各种错误

## 导入各种包

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import to_categorical
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

%matplotlib inline

## 读取数据

这是关于王力宏的歌词excel文件，包含91首歌，共2列属性：歌曲名（Title），歌词（Lyrics）

In [ ]:
file_path = '../input/wanglihong.xlsx'
songs = pd.read_excel(file_path)
print(songs.shape)
songs.head()

查看歌词内容

In [ ]:
for i in range(5):
    print(i, '\n', songs['Lyrics'][i])

## 正则表达式

* ### 以第三首歌为例

英文，符号不要

In [ ]:
song = re.sub(r"[a-zA-Z()''…?.，!！,-]+", '', songs['Lyrics'][3])
song

空格太多，不要！只留一个空格

In [ ]:
re.sub('\s{2,}', ' ', song)

很好，写成函数形式

In [ ]:
def regex_func(text):
    text = re.sub(r"[a-zA-Z()''…?.，!！,-]+", '', text)
    text = re.sub('\s{2,}', ' ', text)
    return text

## 新建一个DataFrame

In [ ]:
new_songs = pd.DataFrame(columns=songs.columns)
new_songs['Title'] = songs['Title']

将正则函数用于歌词，并且添加一个包含歌词长度的属性

In [ ]:
length = []
for i in range(len(new_songs)):
    new_songs.loc[i]['Lyrics'] = regex_func(songs['Lyrics'][i])
    length.append(len(new_songs['Lyrics'][i]))
new_songs['Length'] = length
new_songs.head()

现在拥有一个较干净的数据了，接下来对其搞 · 事 · 情

## 输入输出1

取固定长度为10的输入，这里是输入字符串“秦时明月汉时关 万里”，然后输出“长”；输入“时明月汉时关 万里长”，然后输出“征”，这就是所谓的滑窗(Sliding Window)，如下图：

|输入|输出|
|--|--|
|秦时明月汉时关 万里|长|
|时明月汉时关 万里长|征|
|明月汉时关 万里长征|人|
|月汉时关 万里长征人|未|
|汉时关 万里长征人未|还|

接下来将汉字转化为机器认识的数值类型

# 序列化

In [ ]:
n = len(new_songs) # 歌曲总数目
print('一共有{}首歌'.format(n))

将所有歌词合并成一个长长的字符串

In [ ]:
texts = ''
for i in range(n):
    texts += new_songs['Lyrics'][i]
print('所有歌词的总长度:', len(texts))

## 使用Keras

使用Tokenizer()将文本转换为序列

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

每个字（当然不止汉字，可能还有其它奇奇怪怪的符号，以下统称汉字）对应的下标，返回字典形式——汉字：下标

In [ ]:
tokenizer.word_index # 返回按字频降序排序的字典 {'的': 1, '我': 2, '你': 3, '不': 4, '是': 5, '一': 6, '爱': 7, ...}

每个字的出现频率频，返回字典形式——汉字：频率。以下简称：字频

In [ ]:
tokenizer.word_counts # 返回 OrderedDict([('秦', 2),('时', 93),('明', 62),('月', 35),('汉', 4),('关', 17), ...])

查看字典的长度

In [ ]:
print('字典的长度：', len(tokenizer.word_counts))

使用texts_to_sequences将汉字映射为整数序列

In [ ]:
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
# print('"', texts[: 10],'"',  '分别被映射成了数字：')
print('"{}"{}'.format(texts[: 10], '分别被映射成了整数：'))
print(sequences[: 10])

tokenizer()中提供了参数num_words，在使用时，取字频最多的前(num_words-1)个汉字，多余的为空集，但字典的长度还是和原来的一样

In [ ]:
num_words = 400
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(texts)
print('字典的长度：', len(tokenizer.word_index))

In [ ]:
sequences = tokenizer.texts_to_sequences(texts)
print('"{}"{}'.format(texts[: 10], '分别被映射成了整数：'))
print(sequences[: 10])

对比上面两个sequences的结果来看，第二个sequences里出现了更多的空列表，这就代表了num_words发挥了作用  


### Pad_sequences

> pad_sequences(sequences, maxlen=None, padding='pre', truncating='pre', value=0.0)
* sequences: 列表的列表，每一个元素是一个序列。
* maxlen: 整数，所有序列的最大长度。
* padding: 字符串，'pre' 或 'post' ，在序列的前端补齐还是在后端补齐。
* truncating: 字符串，'pre' 或 'post' ，移除长度大于 maxlen 的序列的值，要么在序列前端截断，要么在后端。
* value: 浮点数，表示用来补齐的值。

In [ ]:
sequences = pad_sequences(sequences)
sequences[: 10]

In [83]:
print('最小索引：{}，最大索引：{}'.format(sequences.min(), sequences.max()))

最小索引：0，最大索引：399


于是sequences里的索引对应了num_words=400

## 新增列

这一列是将歌词映射为整数序列后的新的一列

In [ ]:
star = 0
end = 0
new_songs['Sequences'] = ''
for i in range(n):
    end += new_songs['Length'][i]
    new_songs.loc[i, 'Sequences'] = sequences[star: end]
    star = end
new_songs.head()

In [ ]:
new_songs['Sequences'][0][: 15]

## 输入输出2

这时候拥有了数值型的数据，输入一个序列，输出下一个值：

|输入|输出|
|--|--|
|[0, 46, 87, 171, 0, 46, 341, 0, 220, 31]|[212]|
|[46, 87, 171, 0, 46, 341, 0, 220, 31, 212]|[0]|
|[87, 171, 0, 46, 341, 0, 220, 31, 212, 0]|[15]|
|[171, 0, 46, 341, 0, 220, 31, 212, 0, 15]|[127]|
|[0, 46, 341, 0, 220, 31, 212, 0, 15, 127]|[43]|

接下来操作实现得到上表的数据

## 大序列中的小序列

每首歌之间的歌词是没有联系的，所以每首歌在预测时，并不会从一首歌预测到另一首歌的歌词，不能直接使用texts来滑窗划分

* ### 以第一首歌为例

In [ ]:
sequence = new_songs['Sequences'][0]
sequence

In [ ]:
max_len = 10 # 小序列长度
len_lrc = new_songs['Range'][0] # 歌词长度
X = []
y = []
for i in range(len_lrc - (max_len+1)):
    X.append(sequence[i: i + (max_len+1)])
    y.append(sequence[i + (max_len+1)])

In [ ]:
X
# 返回
# [[0, 46, 87, 171, 0, 46, 341, 0, 220, 31, 212],
# [46, 87, 171, 0, 46, 341, 0, 220, 31, 212, 0],
# [87, 171, 0, 46, 341, 0, 220, 31, 212, 0, 15],
# [171, 0, 46, 341, 0, 220, 31, 212, 0, 15, 127],
# [0, 46, 341, 0, 220, 31, 212, 0, 15, 127, 43],
# ...]

In [ ]:
y # 返回[0, 15, 127, 43,0 , 142, ...]

### 函数形式

In [ ]:
# 这里将X，y合并成一个大矩阵，那么大矩阵的shape=(?, 11)
def build_matrix(sequence, max_len = 10):
    max_len += 1
    matrix = []
    length = len(sequence)
    for i in range(length - max_len):
        matrix.append(sequence[i: i + max_len])
    matrix = np.array(matrix)
    X = matrix[:, :-1]
    y = matrix[:, -1]
    return X, y

### 序列合并

In [ ]:
# n = 91 歌曲数目
X, y = build_matrix(new_songs['Sequences'][0])
for i in range(1, n):
    sequence = new_songs['Sequences'][i]
    XX, yy = build_matrix(sequence)
    X = np.concatenate([X, XX])
    y = np.concatenate([y, yy])

In [ ]:
X
# 返回
# array([[  0,  46,  87, ...,   0, 220,  31],
#        [ 46,  87, 171, ..., 220,  31, 212],
#        [ 87, 171,   0, ...,  31, 212,   0],
#        ...,
#        [  3,   0, 268, ...,  52,  52,   1],
#        [  0, 268,  49, ...,  52,   1,   0],
#        [268,  49,   0, ...,   1,   0, 268]])

In [ ]:
y # 返回 array([212,   0,  15, ...,   0, 268,  49])

In [ ]:
X.shape, y.shape

一共得到32574行的输入序列

## 输入输出3

将输入序列进行字向量化（此处为单个汉字，故称字向量，而使用更多的为词向量），输出进行One hot编码，例如（数值是瞎写的）：

|整数|One Hot|字向量|
|--|--|--|
|3|[0, 0, 0, 1, 0, 0]|[0.32, 0.11]|
|6|[1, 0, 0, 0, 0, 0]|[0.51, 0.62]||
|4|[0, 0, 1, 0, 0, 0]|[0.26, 0.41]|

## 建立模型


在Keras里面，使用嵌入层 Embedding 可以输入进行向量化

> Embedding(input_dim, out_dim, input_length=None)  
* input_dim:大或等于0的整数，字典长度，即输入数据最大下标+1
* out_dim:大于0的整数，代表全连接嵌入的维度  
* input_length:当输入序列的长度固定时，该值为其长度

现在可以举个例子，拿 X[: 1] 来做示范

In [80]:
print('X[: 1]是', X[: 1].shape, '矩阵')
print('它含有数值：', X[: 1])

X[: 1]是 (1, 10) 矩阵
它含有数值： [[  0  46  87 171   0  46 341   0 220  31]]


Embedding 层 input_dim 就是汉字表的数目 num_words，out_dim 就是我们需要让整数向量化后的长度，input_length 就是输入序列的长度，我们的 X.shape[1] = 10，就是10，例如我们这么写：

In [ ]:
embedding = Sequential()
embedding.add(Embedding(num_words, 12, input_length=X.shape[1]))

这样，输入的 X[: 1].shape = (1, 10) 就会变为 (1, 10, 12)

In [82]:
print(embedding.predict(X[: 1]))
print(embedding.predict(X[: 1]).shape)

[[[ 0.00410016 -0.04466455 -0.00399476  0.01623822 -0.04920522
   -0.00889667 -0.01161783 -0.00897527 -0.01439846 -0.01946389
   -0.02629197  0.04435097]
  [-0.02194022  0.00509825  0.0128585   0.00029064 -0.00293522
    0.00266709  0.00053818 -0.02969973 -0.03511238  0.02033797
    0.02721104  0.0055184 ]
  [-0.04881546 -0.04140599 -0.04774748  0.02519364 -0.00897771
   -0.02298084 -0.0318353  -0.01423757  0.01061138 -0.00734457
   -0.00064385 -0.04373505]
  [-0.02437471 -0.04726074 -0.04840286 -0.010079    0.00221704
   -0.03893149 -0.03813889 -0.01741491 -0.01908786 -0.02397026
   -0.02031325  0.01271491]
  [ 0.00410016 -0.04466455 -0.00399476  0.01623822 -0.04920522
   -0.00889667 -0.01161783 -0.00897527 -0.01439846 -0.01946389
   -0.02629197  0.04435097]
  [-0.02194022  0.00509825  0.0128585   0.00029064 -0.00293522
    0.00266709  0.00053818 -0.02969973 -0.03511238  0.02033797
    0.02721104  0.0055184 ]
  [ 0.01512449 -0.02926758  0.00187331  0.03212029 -0.00457566
    0.0476350

从上面看来，即  
0 -> [ 0.00410016 -0.04466455 -0.00399476  0.01623822 -0.04920522 -0.00889667 -0.01161783 -0.00897527 -0.01439846 -0.01946389 -0.02629197  0.04435097]  
46 -> [-0.02194022  0.00509825  0.0128585   0.00029064 -0.00293522  0.00266709  0.00053818 -0.02969973 -0.03511238  0.02033797  0.02721104  0.0055184 ]
...

这样就可以马上开工了！

输出层的节点数为num_words，即400个，使用softmax激活函数，然后获取最大输出值的下标，寻找字典中对应的汉字

损失函数使用categorical_crossentropy，优化函数使用adam  
> 当使用 categorical_crossentropy 损失时，你的目标值应该是分类格式 (即，如果你有 10 个类，每个样本的目标值应该是一个 10 维的向量，这个向量除了表示类别的那个索引为 1，其他均为 0)。

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 128, input_length=X.shape[1]))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

使用 to_categorical() 对 y 进行One hot编码

In [ ]:
y = to_categorical(y, num_classes=num_words)

### 开始训练

In [ ]:
model.fit(X, y, batch_size=258, epochs=500, verbose=0, )
model.save('lstm_model.h5')

## 可视化

In [ ]:
loss = model.history.history['loss']

plt.style.use('bmh')
plt.figure(figsize=(12, 8))
plt.plot(range(len(loss)), loss)
plt.title('LSTM')
plt.xlabel('Iterations')
plt.ylabel('Loss')
# plt.savefig('lstm_0.png')

In [ ]:
# a = 200
# b = 800
# loss_ = loss[a: b]
# plt.figure(figsize=(12, 8))
# plt.plot(range(a, b), loss_)
# plt.title('LSTM')
# plt.xlabel('Iterations')
# plt.ylabel('Loss')

## 预测

In [ ]:
test_lrc = '当节奏开始转' # 输入开头
test_sequence = tokenizer.texts_to_sequences(test_lrc) # 序列化
test_sequence = pad_sequences(test_sequence).reshape(1, -1)
test_sequence = pad_sequences(test_sequence, X.shape[1]) # 输入序列长度不足10，故使用pad_sequences将其补足
test_sequence

In [ ]:
print('输出的最大下标为：', model.predict(test_sequence).argmax())

Tokenizer()中的字典是以1开始作为下标的，故我们将下标0看作是空格

In [ ]:
try:
    print(tokenizer.index_word[0])
except:
    print('字典中没有')

我们需要要让模型不断的输出，就要有一个不断输入的循环，将输出添加到原来的输入当中，于是就有了新的输入

In [ ]:
test_lrc += ' ' # 下标为0，故添加一个空格
test_lrc

新的汉字加进来后

In [ ]:
test_sequence = tokenizer.texts_to_sequences(test_lrc)
test_sequence = pad_sequences(test_sequence).reshape(1, -1)
test_sequence = pad_sequences(test_sequence, X.shape[1])
test_sequence

### 函数形式

In [ ]:
# 输入一个文本将其转化为序列
def input_sequence(text, max_len=10):
    sequence = tokenizer.texts_to_sequences(text) # 序列化
    sequence = pad_sequences(sequence).reshape(1, -1) # 填充0
    sequence = pad_sequences(sequence, maxlen=max_len) # 补足或截断
    return sequence

In [ ]:
def next_word(y_pred):
    idx = np.argmax(y_pred) # 最大值的下标
    if idx == 0: # 下标为0时，字典内并不存在，故视为空格
        return ' '
    else:
        return tokenizer.index_word[idx]

输出长度为200的歌词试试看：

In [ ]:
lrc = '当节奏开始转'
for i in range(200):
    X_sequence = input_sequence(lrc)
    y_pred = model.predict(X_sequence)
    word = next_word(y_pred)
    lrc += word
lrc = re.sub('\s+', ' ', lrc) # 只需要一个空格
print(lrc)

然而我们并没有看懂他在讲什么

### 函数形式

In [ ]:
def generating_lrc(lrc, length=200):
    for i in range(200):
        X_sequence = input_sequence(lrc)
        y_pred = model.predict(X_sequence)
        word = next_word(y_pred)
        lrc += word
    lrc = re.sub('\s+', ' ', lrc) # 只需要一个空格
    return lrc

In [ ]:
generating_lrc('大城小爱')

In [ ]:
generating_lrc('他根本就不会用丹田唱歌')

## 参考资料

1. [Generating Drake Rap Lyrics using Language Models and LSTMs](https://towardsdatascience.com/generating-drake-rap-lyrics-using-language-models-and-lstms-8725d71b1b12)（译文：[使用Keras和LSTM生成说唱歌词](https://www.jqr.com/article/000236)）
2. [序列预处理 - Keras中文文档](https://keras.io/zh/preprocessing/sequence/)
3. [嵌入层 Embedding - Keras中文文档](https://keras-cn.readthedocs.io/en/latest/layers/embedding_layer/)
4. [损失函数 Losses - Keras中文文档](https://keras.io/zh/losses/)
5. [深度学习中Keras中的Embedding层的理解与使用](http://frankchen.xyz/2017/12/18/How-to-Use-Word-Embedding-Layers-for-Deep-Learning-with-Keras/)